# Hybrid VAE

We build a hybrid VAE that uses a quantum circuit to encode the latent space, and a classical neural network to decode it. 

In [1]:
import pennylane as qml
from pennylane import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim


# settings
n_qubits = 2
n_layers = 1
dev = qml.device("default.qubit", wires=n_qubits)

## Quantum Encoder

We define a quantum circuit that encodes the input features into a quantum state.

The circuit interfaces with PyTorch, allowing us to use it as part of a larger neural network.

In [2]:
@qml.qnode(dev, interface="torch")
def quantum_encoder(inputs, weights):
    # Encoding input features as rotations
    for i in range(n_qubits):
        qml.RY(inputs[i], wires=i)
    
    # Variational layers
    for l in range(n_layers):
        for i in range(n_qubits):
            qml.RY(weights[l, i, 0], wires=i)
            qml.RZ(weights[l, i, 1], wires=i)
        qml.CNOT(wires=[0, 1])

    # Measurement
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

This is a PyTorch Module that internally uses the quantum encoder defined above. 

In [3]:
class QuantumEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        # Variational parameters (torch Parameter)
        self.q_weights = nn.Parameter(0.01 * torch.randn(n_layers, n_qubits, 2))

    def forward(self, x):
        batch_out = []
        for i in range(x.shape[0]):
            res = quantum_encoder(x[i], self.q_weights)
            # Force dtype to float32
            res_torch = torch.tensor(res, dtype=torch.float32)
            batch_out.append(res_torch)
        return torch.stack(batch_out)

## Classical Decoder

This is a classical neural network that decodes the latent space into the output space.

In [4]:
class ClassicalDecoder(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 8),
            nn.ReLU(),
            nn.Linear(8, output_dim)
        )

    def forward(self, z):
        return self.model(z)

## Hybrid VAE

Finally, we combine the quantum encoder and classical decoder into a hybrid VAE model.

In [5]:

class HybridVAE(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = QuantumEncoder()
        self.decoder = ClassicalDecoder(latent_dim=n_qubits, output_dim=2)

    def forward(self, x):
        z = self.encoder(x)
        x_recon = self.decoder(z)
        return x_recon

## Training with PyTorch

In [6]:

# Synthetic data generation
X_data = torch.rand((100, 2), dtype=torch.float32) * np.pi

# Training the Hybrid VAE
model = HybridVAE()
opt = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

for epoch in range(30):
    x_recon = model(X_data)
    loss = loss_fn(x_recon, X_data)

    opt.zero_grad()
    loss.backward()
    opt.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Epoch 1, Loss: 3.5962
Epoch 2, Loss: 3.4688
Epoch 3, Loss: 3.3442
Epoch 4, Loss: 3.2215
Epoch 5, Loss: 3.1004
Epoch 6, Loss: 2.9807
Epoch 7, Loss: 2.8624
Epoch 8, Loss: 2.7455
Epoch 9, Loss: 2.6299
Epoch 10, Loss: 2.5155
Epoch 11, Loss: 2.4025
Epoch 12, Loss: 2.2909
Epoch 13, Loss: 2.1806
Epoch 14, Loss: 2.0719
Epoch 15, Loss: 1.9647
Epoch 16, Loss: 1.8593
Epoch 17, Loss: 1.7557
Epoch 18, Loss: 1.6540
Epoch 19, Loss: 1.5545
Epoch 20, Loss: 1.4574
Epoch 21, Loss: 1.3629
Epoch 22, Loss: 1.2709
Epoch 23, Loss: 1.1816
Epoch 24, Loss: 1.0952
Epoch 25, Loss: 1.0125
Epoch 26, Loss: 0.9341
Epoch 27, Loss: 0.8603
Epoch 28, Loss: 0.7913
Epoch 29, Loss: 0.7275
Epoch 30, Loss: 0.6694
